### Import Required Libraries and Set Up Environment Variables

In [330]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [331]:
# Set environment variables from the .env in the local environment
load_dotenv('.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [333]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=fJJgTV2TPG63TevU9rJB2kV79CNQ7aUK


In [334]:
# Create an empty list to store the reviews
reviews_list= []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_url)
    reviews = response.json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f'Checked page {page}')
    except:
        # Print the page number that had no results then break from the loop
        print(f'Checked page {page}, no results')
        break

Checked page 0


In [335]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

[{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html',
  'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.',
  'source': 'The New York Times',
  'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick',
   'kicker': None,
   'content_kicker': None,
   'print_headline': 'The Attachment Diaries',
   'name': None,
   'seo': None,
   'sub': None},
  'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'},
   {'name': 'creative_works',
    'value': 'The Attachment Diaries (Movie)',
    'rank': 2,
    'major': 'N'},
   {'name': 'persons',
    'value': 'Diment, Valentin Javier',
    'rank': 3,
    'major': 'N'}],
  'pub_date': '2023-05-25T11:00:03+0000',
  'byline': {'original': 'By Jeannette Catsoulis',
   'person': [{'firstname': 'Jeannette',
     'middlename': None,
     'lastname': 'Catsoulis',
     'qualifier': None,
     'title': None,
     'rol

In [336]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [337]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df_reviews['title'] = df_reviews['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

In [338]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)

In [339]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = df_reviews['title'].to_list()
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Ti

### Access The Movie Database API

In [341]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [342]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Sleep through Rate Limit... Don't Get Caught!")
        time.sleep(10)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_url = f"{url}{title}{tmdb_key_string}"
    search_response = requests.get(search_url)
    search_data = search_response.json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = search_data['results'][0]['id']

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_details = movie_response.json()

        # Execute "GET" request with url
        movie_response = requests.get(movie_url)
        movie_details = movie_response.json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_dict = {
            "title": title,
            "original_title": movie_details['original_title'],
            "budget": movie_details['budget'],
            "original_language": movie_details['original_language'],
            "homepage": movie_details['homepage'],
            "overview": movie_details['overview'],
            "popularity": movie_details['popularity'],
            "runtime": movie_details['runtime'],
            "revenue": movie_details['revenue'],
            "release_date": movie_details['release_date'],
            "vote_average": movie_details['vote_average'],
            "vote_count": movie_details['vote_count'],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }
        tmdb_movies_list.append(movie_dict)
        
        # Print out the title that was found
        print(f"Found details for: {title}")
    except:
        print(f"Details not found for: {title}")

Found details for: The Attachment Diaries
Details not found for: What’s Love Got to Do With It?’ Probably a Lo
Found details for: You Can Live Forever
Found details for: A Tourist’s Guide to Love
Found details for: Other People’s Children
Found details for: One True Loves
Found details for: The Lost Weekend: A Love Story
Found details for: A Thousand and One
Found details for: Your Place or Mine
Found details for: Love in the Time of Fentanyl


In [343]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.789,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [344]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.789,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.621,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",13.948,96,0,2023-04-21,6.289,166,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",26.688,104,84178,2022-09-21,6.814,191,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,15.627,100,37820,2023-04-07,6.513,77,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,30.663,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]
6,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,13.474,116,3395595,2023-03-31,6.871,89,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]
7,Your Place or Mine,Your Place or Mine,0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,20.817,109,0,2023-02-10,6.282,725,"[Romance, Comedy]",[English],[United States of America]
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,en,,An intimate portrait of a community fighting t...,1.763,85,0,2023-02-03,0.000,0,[Documentary],[English],"[Canada, United States of America]"


### Merge and Clean the Data for Export

In [346]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, tmdb_df, on='title', how='inner')

In [347]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df.head())

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/21/movies/a-to...   
3  https://www.nytimes.com/2023/04/20/movies/othe...   
4  https://www.nytimes.com/2023/04/13/movies/one-...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
3  A radiant Virginie Efira stars as a Parisian t...  The New York Times   
4  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;creative_works: You Can Live F...   
2  subject: Movies;creative_works: A T

In [348]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)

In [349]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
print(merged_df.head())

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/21/movies/a-to...   
3  https://www.nytimes.com/2023/04/20/movies/othe...   
4  https://www.nytimes.com/2023/04/13/movies/one-...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
3  A radiant Virginie Efira stars as a Parisian t...  The New York Times   
4  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;creative_works: You Can Live F...   
2  subject: Movies;creative_works: A T

In [350]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)